# skype pair permutations 1


In [ ]:
%config InteractiveShellApp.pylab_import_all = False
%matplotlib inline
%pylab inline
%reload_ext autoreload
%autoreload 2

import sys
import os
import fnmatch

import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame, Series
import seaborn as sns
import glob
from datetime import datetime
from scipy.optimize import curve_fit
from scipy import stats

propsFn='props.csv'
props=pd.read_csv(propsFn, header=None, index_col=0, squeeze=True,delim_whitespace=True).to_dict()

base=props['BaseDir']
expFile=props['allExpFn']

RawDataDir = os.path.join(base,props['RawDataDir'])+'\\'
ProcessingDir = os.path.join(base,props['ProcessingDir'])+'\\Fig4BC_skypePermutation\\'
outputDir = os.path.join(base,props['outputDir'])+'\\'

if not os.path.isdir(ProcessingDir):
    os.makedirs(ProcessingDir)
if not os.path.isdir(outputDir):
    os.makedirs(outputDir)

os.chdir('..\\')
import functions.matrixUtilities_joh as mu
import matplotlib.pyplot as plt
import models.experiment as xp
import models.experiment_set as es
import functions.paperFigureProps as pfp
pfp.paper()
inToCm=2.54


In [ ]:
info=pd.read_csv(expFile, sep=',')
info=info[info.stimulusProtocol=='sp']
info

In [ ]:
# collect meta information and save to new csv file for batch processing

aviPath=[]
posPath=[]
PLPath=[]
expTime = []
    
for index,row in info.iterrows():
    startDir=RawDataDir+row.path+'\\'
    #startDir='D:\\data\\b\\2017\\'+row.path+'\\'
    #if not os.path.isdir(startDir):
    #    startDir='E:\\b\\2017\\'+row.path+'\\'
        
    posPath.append(glob.glob(startDir+'PositionTxt*.txt')[0])
    PLPath.append(glob.glob(startDir+'PL*.txt')[0])
    
    head, tail = os.path.split(posPath[-1])
    currTime=datetime.strptime(tail[-23:-4], '%Y-%m-%dT%H_%M_%S')
    expTime.append(currTime)
    
info['txtPath']=posPath
info['pairList']=PLPath

info['epiDur'] = 5      # duration of individual episodes (default: 5 minutes)
info['episodes'] = -1   # number of episodes to process: -1 to load all episodes (default: -1)
info['inDish'] = 10#np.arange(len(posPath))*120     # time in dish before experiments started (default: 10)
info['arenaDiameter_mm'] = 100 # arena diameter (default: 100 mm)
info['minShift'] = 60 # minimum number of seconds to shift for control IAD
info['episodePLcode'] = 1 # flag if first two characters of episode name encode animal pair matrix (default: 0)
info['recomputeAnimalSize'] = 1 # flag to compute animals size from avi file (takes time, default: 1)
info['SaveNeighborhoodMaps'] = 0 # flag to save neighborhood maps for subsequent analysis (takes time, default: 1)
info['computeLeadership'] = 0 # flag to compute leadership index (takes time, default: 1)
info['ComputeBouts'] = 0 # flag to compute swim bout frequency (takes time, default: 1)
info['set'] = np.arange(len(posPath))   # experiment set: can label groups of experiments (default: 0)
info['ProcessingDir']=ProcessingDir
info['outputDir']=outputDir
info['allowEpisodeSwitch']=1

info['expTime']=expTime

csvFile=os.path.join(ProcessingDir,'Fig4BC_skypePermutation.csv')
info.to_csv(csvFile,encoding='utf-8')
info

In [ ]:
def readExperiment(keepData=False):
    tmp=es.experiment_set(csvFile=csvFile)
    if keepData:
        return tmp
    else:
        return 1

expSet=readExperiment(keepData=True)

In [ ]:
csvPath = []
for f in [mu.splitall(x)[-1][:-4] for x in info.txtPath]:
    csvPath.append(glob.glob(ProcessingDir+f+'*siSummary*.csv')[0])

df=pd.DataFrame()

for fn in csvPath:
    print(fn)
    tmp=pd.read_csv(fn,index_col=0,sep=',')
    df=pd.concat([df,tmp])

df['episode']=[x.strip().replace('_','') for x in df['episode']]
print('df shape',df.shape)

d=df.time
r=datetime(2017,1,1)
t2=[pd.to_datetime(x).replace(day=1,month=1)for x in df.time]

t3=[(x-r)/pd.Timedelta('1 hour') for x in t2]
df['t2']=t2
df['t3']=t3
df.head()

Pre-Analyze all experiments only if necessary, this takes a couple of minutes! Experiment summary csv files are saved to disk.

## Habituation or Fatigue within 20 hours?

Plot shoaling index during closed loop skype episodes over time.

In [ ]:
sns.tsplot(data=df, time="t3",value="si",unit="animalIndex",condition="episode",estimator=np.nanmean,interpolate=False,err_style="ci_bars");
plt.xlim([0,24])

In [ ]:
PL=np.loadtxt(PLPath[0]).astype('int')
PL

In [ ]:
partnerList=[]
for index,row in df.iterrows():
    pairListNr=int(row.episode[:2])
    pairList=PL[pairListNr*16:(pairListNr+1)*16]
    cp=np.where(pairList[:,row.animalIndex])[0][0]
    partnerList.append(cp)
    
cp=np.array(partnerList)
df['partnerIndex']=partnerList
df.head()

In [ ]:
pfp.paper()
inToCm=2.54
plt.figure(figsize=(12/inToCm,6/inToCm))
ax = plt.gca()

plotTime=6*5*15
print('plotting minutes 0 to ',plotTime)
print('plotting episodes up to', plotTime/5)
idx=(df['inDishTime']<plotTime) & (df['inDishTime']>0)
SIAll_si=df[idx].groupby(['animalIndex','partnerIndex']).mean().si.values#.reset_index()#.values.reshape(15,-1)
SIAll_sd=df[idx].groupby(['animalIndex','partnerIndex']).std().si.values#.reset_index()#.values.reshape(15,-1)


SIAll_si_half=np.squeeze(np.triu(SIAll_si.reshape(15,-1),k=1).reshape(-1,1))

order=np.argsort(SIAll_si_half)[::-1]
order=order[:105]
plt.plot(SIAll_si_half[order],'.k')
plt.errorbar(range(order.shape[0]),SIAll_si_half[order],yerr=SIAll_sd[order],color='gray')
plt.xlim([-1,105])
plt.yticks([0,.5])
plt.axhline(0,ls=':',color='k')
plt.xlabel('Pair #')
plt.ylabel('Attraction')
sns.despine()


In [ ]:

idx=(df['inDishTime']<plotTime) & (df['inDishTime']>0)


siMat1=np.zeros((15,16))*np.nan
siMat2=np.zeros((15,16))*np.nan
print(siMat2.shape)
for i in range(15):
    tmp1=df[idx][df[idx].animalIndex==i].groupby(['partnerIndex']).mean().si.reset_index()
    tmp2=df[idx][df[idx].animalIndex==i].groupby(['partnerIndex']).std().si.reset_index()
    siMat1[i,tmp1.partnerIndex.values]=tmp1.si.values
    siMat2[i,tmp2.partnerIndex.values]=tmp2.si.values
    

pfp.paper()
inToCm=2.54
plt.figure(figsize=(12/inToCm,6/inToCm))
ax = plt.gca()

plotTime=6*5*15
print('plotting minutes 0 to ',plotTime)
print('plotting episodes up to', plotTime/5)


SIAll_si_half=np.squeeze(np.tril(siMat1,k=-1).reshape(-1,1))
SIAll_std_half=np.squeeze(np.tril(siMat2,k=-1).reshape(-1,1))


order=np.argsort(SIAll_si_half)[::-1]
order=order[:105]
plt.plot(SIAll_si_half[order],'.k')
plt.errorbar(range(order.shape[0]),SIAll_si_half[order],yerr=SIAll_std_half[order],color='gray')
plt.xlim([-1,105])
plt.yticks([0,.5])
plt.axhline(0,ls=':',color='k')
plt.xlabel('Pair #')
plt.ylabel('Attraction')
sns.despine()
#plt.text(-15,0.8,'Figure S6',fontsize=12,fontweight='bold')
figPath=outputDir+'\\S6_PairPermutationSTD.svg'
plt.savefig(figPath)


In [ ]:
plotTime=6*5*15
idx=(df['inDishTime']<plotTime) & (df['inDishTime']>0)


siMat=np.zeros((15,16))*np.nan
print(siMat.shape)
for i in range(15):
    tmp=df[idx][df[idx].animalIndex==i].groupby(['partnerIndex']).mean().si.reset_index()
    siMat[i,tmp.partnerIndex.values]=tmp.si.values

ax=plt.imshow(siMat,interpolation='None')
plt.grid(False)

In [ ]:
xSort=np.argsort(np.nansum(siMat[:,:-1],axis=0))[::-1]
xSort=np.hstack([xSort,15])
xSort

In [ ]:
siMats=siMat[xSort[:-1],:]
siMatss=siMats[:,xSort]
fig, ax = plt.subplots()
im=ax.imshow(siMatss,interpolation='None',cmap='viridis')
plt.grid(False)
plt.xticks(range(16),xSort);
plt.yticks(range(15),xSort[:-1]);
ax.xaxis.tick_top()
plt.colorbar(im)

In [ ]:
xSort2=np.argsort(siMat[:,-1])[::-1]
xSort2=np.hstack([xSort2,15])
xSort2


In [ ]:
siMats2=siMat[xSort2[:-1],:]
siMatss2=siMats2[:,xSort2]
fig, ax = plt.subplots()
im=ax.imshow(siMatss2,interpolation='None',cmap='viridis')
plt.grid(False)
plt.xticks(range(16),xSort2);
plt.yticks(range(15),xSort2[:-1]);
ax.xaxis.tick_top()
plt.colorbar(im)

In [ ]:
from matplotlib import gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
img=siMatss
pfp.paper()
inToCm=2.54

fig=plt.figure(figsize=(6.0/inToCm,5.5/inToCm))
t = np.arange(img.shape[1])
f = np.arange(img.shape[0])
flim = (np.nanmin(f)-.5, np.nanmax(f)+.5)
tlim = (np.nanmin(t)-1, np.nanmax(t)+1)

gs = gridspec.GridSpec(2, 2, width_ratios=[5,1], height_ratios=[1.5,5])

ax = fig.add_subplot(gs[1,0])
axl = fig.add_subplot(gs[1,1], sharey=ax)
axb = fig.add_subplot(gs[0,0], sharex=ax)
gs.update(hspace=-.1, wspace=-.1)
im=ax.imshow(img, origin='upper',interpolation='nearest',aspect='equal',cmap='viridis',clim=(0.0, 0.5))
axl.fill_betweenx(f,img[:,-1],color='gray')
axl.plot(img[:,-1], f,'.',color='k')
axb.fill_between(t, 0,np.nanmean(img,0),color='gray')
axb.plot(t, np.nanmean(img,0),'.',color='k')#ax.set_xlim(tlim)
axb.plot(t[-1], np.nanmean(img,0)[-1],'.',color='r')#ax.set_xlim(tlim)
ax.axvline(14.5,ls='--',color='r',lw=2)
axb.axvline(14.5,ls='--',color='r',lw=2)
axb.set_ylim([-.2,.6])
axl.set_xlim([-0.05,.6])

ax.set_ylim(flim[::-1])
ax.set_xlim([-1.5,16.5])
axb.set_xlim([-1.5,16.5])
ax.grid(b=False)
axb.axis('off')
#axl.axis('off')
axl.set_ylabel('Unilateral attraction',rotation=270, labelpad=10)
axl.yaxis.set_label_position("right")

for axe in [ax,axl,axb]:
    plt.setp(axe.get_yticklabels(), visible=False)
    plt.setp(axe.get_xticklabels(), visible=False)
    axe.set_xticks([])
    axe.set_yticks([])
    axe.spines['left'].set_color('white')
    axe.spines['top'].set_color('white')
    axe.spines['right'].set_color('white')
    axe.spines['bottom'].set_color('white')
    axe.tick_params(axis='y', which='both',length=0)
    axe.tick_params(axis='x', which='both',length=0)

fig.text(0.1, .35, 'Animal 1-15', va='center',fontsize=10,rotation=90)
fig.text(0.35, .05, 'Animal 1-15', ha='center',fontsize=10)

fig.text(0.4, .9, 'Mean bilateral attraction', ha='center',fontsize=10)
plt.subplots_adjust(wspace=-.2, hspace=-.1)
#fig.set_size_inches(7/inToCm,4.5/inToCm)


In [ ]:
from matplotlib import gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
img=siMatss2
pfp.paper()
inToCm=2.54

fig=plt.figure(figsize=(6.0/inToCm,5.5/inToCm))
t = np.arange(img.shape[1])
f = np.arange(img.shape[0])
flim = (np.nanmin(f)-.5, np.nanmax(f)+.5)
tlim = (np.nanmin(t)-1, np.nanmax(t)+1)

gs = gridspec.GridSpec(2, 2, width_ratios=[5,1], height_ratios=[1.5,5])

ax = fig.add_subplot(gs[1,0])
axl = fig.add_subplot(gs[1,1], sharey=ax)
axb = fig.add_subplot(gs[0,0], sharex=ax)
gs.update(hspace=-.1, wspace=-.1)
im=ax.imshow(img, origin='upper',interpolation='nearest',aspect='equal',cmap='viridis',clim=(0.0, 0.5))
axl.fill_betweenx(f,img[:,-1],color='gray')
axl.plot(img[:,-1], f,'.',color='k')
axb.fill_between(t, 0,np.nanmean(img,0),color='gray')
axb.plot(t, np.nanmean(img,0),'.',color='k')#ax.set_xlim(tlim)
#axb.plot(t[-1], np.nanmean(img,0)[-1],'.',color='r')#ax.set_xlim(tlim)
ax.axvline(14.5,ls='--',color='r',lw=2)
axb.axvline(14.5,ls='--',color='r',lw=2)
axb.set_ylim([-.2,.6])
axl.set_xlim([-0.05,.6])

ax.set_ylim(flim[::-1])
ax.set_xlim([-1.5,16.5])
axb.set_xlim([-1.5,16.5])
ax.grid(b=False)
axb.axis('off')
#axl.axis('off')
axl.set_ylabel('Unilateral attraction',rotation=270, labelpad=10)
axl.yaxis.set_label_position("right")

for axe in [ax,axl,axb]:
    plt.setp(axe.get_yticklabels(), visible=False)
    plt.setp(axe.get_xticklabels(), visible=False)
    axe.set_xticks([])
    axe.set_yticks([])
    axe.spines['left'].set_color('white')
    axe.spines['top'].set_color('white')
    axe.spines['right'].set_color('white')
    axe.spines['bottom'].set_color('white')
    axe.tick_params(axis='y', which='both',length=0)
    axe.tick_params(axis='x', which='both',length=0)

fig.text(0.1, .35, 'Animal 1-15', va='center',fontsize=10,rotation=90)
fig.text(0.35, .05, 'Animal 1-15', ha='center',fontsize=10)

fig.text(0.4, .9, 'Mean bilateral attraction', ha='center',fontsize=10)
plt.subplots_adjust(wspace=-.2, hspace=-.1)
#fig.set_size_inches(7/inToCm,4.5/inToCm)
figPath=outputDir+'\\4B_pairPermutation_b.svg'
plt.savefig(figPath)

In [ ]:
import scipy.stats
pfp.paper()
inToCm=2.54
plt.figure(figsize=(4.5/inToCm,4.5/inToCm))
ax = plt.gca()

uniLateral=img[:,-1]
biLateral=np.nanmean(img,0)[:-1]
plt.plot(uniLateral,biLateral,'k.-')

sns.despine()

s,i,r,p,std=scipy.stats.linregress(uniLateral,biLateral)
print(s,i,r,p,std)
t=np.linspace(0,.5,100)
l=i+s*t
ax.plot(t,l,ls='--',color='r',lw=2)
ax.text(.1,.5,'R: '+str(r)[:4])
ax.text(.1,.4,"p = {:.0e}".format(p),color='k')
ax.set_ylim([0,.5])
ax.set_xlim([0,.5])

In [ ]:
plt.figure(figsize=(2.5/inToCm,1.5/inToCm))
ax = plt.gca()

cb1=plt.colorbar(im,ax=ax,ticks=[],format='%.1f',
                 orientation='horizontal',
                aspect=4)
#cb1.set_label('Attraction',labelpad=2)
cb1.ax.yaxis.set_ticks_position('right')

ax.remove()
 #cbar.ax.text(.5, (2 * j + 1) / 8.0, lab, ha='center', va='center')
cb1.ax.text(.5, -2, 'Attraction', ha='center',fontsize=10)
cb1.ax.text(-.5, 0, '0.0', ha='center',fontsize=10)
cb1.ax.text(1.5, 0, '0.5', ha='center',fontsize=10)
figPath=outputDir+'\\4Bb_pairPermutation_cbar.svg'
plt.savefig(figPath,bbox_inches='tight')

In [ ]:
simAll=[]
dyad=[]
actAll=[]
simArray=np.zeros((105,4))
n=0
for i in range(15):
    a1=img[i,-1]
    for j in range(i+1,15):
        a2=img[j,-1]
        simAll.append(a1+a2)
        actAll.append(img[j,i])
        simArray[n,:]=a1,a2,a1+a2,img[j,i]
        n+=1
        


In [ ]:
import scipy.stats
pfp.paper()
inToCm=2.54
plt.figure(figsize=(4.5/inToCm,4.5/inToCm))
ax = plt.gca()

ax.scatter(np.array(simAll),np.array(actAll),color='gray',s=6)

#dfSI.plot(kind='scatter',x='physical',y='virtual',ax=ax,c=ageCol[dfSI.age.values.astype('int')])
ax.set_ylim([0,1])
ax.set_xlim([0,1])
ax.set_yticks([0,.5,1])
ax.set_xticks([0,.5,1])
ax.set_ylabel('Observed \n bilateral attraction')
ax.set_xlabel('Linear combination prediction \n (unilateral #1 + #2)')
#ax.set_title('attraction')
sns.despine()

s,i,r,p,std=scipy.stats.linregress(np.array(simAll),np.array(actAll))
print(s,i,r,p,std)
t=np.linspace(0,1,100)
l=i+s*t
ax.plot(t,l,ls='--',color='k',lw=2)
plt.plot(np.arange(0,1,.1),np.arange(0,1,.1),'gray',ls=':')
ax.text(.1,.9,'R: '+str(r)[:4])
ax.text(.1,.8,"p = {:.0e}".format(p),color='k')
ax.text(.1,.7,"slope = {:.2f}".format(s),color='k')
#ax.text(.1,.8,'p: '+str(p)[:1]+str(p)[-4:])

figPath=outputDir+'\\4C_dyadPredictCorr.svg'
plt.savefig(figPath)

# predict Psocial using unilateral model parameters

In [ ]:
csvFileNetLogo=os.path.join(RawDataDir,'20180316_netlogo2Animals\\Shoaling paperRun-table.csv')

df_NL=pd.read_csv(csvFileNetLogo,skiprows=6)
df_NL=df_NL.groupby(['[run number]','t1-Social','t2-Social']).mean().IAD.reset_index() #drop other columns

#calculate shoaling index, using IAD when both agents have Ps<0.01
idx=(df_NL['t1-Social']<0.01) & (df_NL['t2-Social']<0.01) 
IADs=df_NL[idx].IAD
print('mean IAD without attraction: ',IADs.mean())
df_NL['SI']=(IADs.mean()-df_NL.IAD)/IADs.mean()
df_NL.head()

In [ ]:
#Pivot table of group means for all combinations of Ps1 & Ps2 for plotting
dfAvg=df_NL.groupby(['t2-Social','t1-Social']).mean().SI.reset_index()
dfAvgPiv=pd.pivot_table(dfAvg,index='t1-Social',columns='t2-Social',values='SI')
dfAvgPiv

In [ ]:
pfp.paper()
sns.set_palette('viridis',6)

fig, ax = plt.subplots(figsize=(4.5/inToCm,4.5/inToCm))
for i in np.array([0,2,4,5,6,7]): # plot subset to avoid crowded plot
    ax.plot(dfAvgPiv.columns,dfAvgPiv.values[i,:],'.-',label=dfAvgPiv.index[i]);

ax.set_xlim([-.05,1.05])    
ax.set_ylim([-.05,1.05])   
sns.despine()
plt.legend(title='Ps1 (agent 1)',ncol=2,
          bbox_to_anchor=(0.15, 0.45, 1., .102),
          handletextpad=0.1);
plt.xlabel('Ps2 (agent 2)')
plt.ylabel('Model attraction')

figPath=outputDir+'\\4E_Model_SocialCombinations.svg'
plt.savefig(figPath)

In [ ]:

def BrodyGrowth(x, a, b, c):
    return a-(a-b)*np.exp(-c*x)

In [ ]:
x = dfAvgPiv.columns
yn = dfAvgPiv.values[0,:]

popt, pcov = curve_fit(BrodyGrowth, x, yn)
poptPs10=popt
print(poptPs10) # should be [ 8.60418613e-01 -5.19998745e-04  4.24005018e+00]

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
ax.plot(x, yn, 'k.', label="model data")
ax.plot(x, BrodyGrowth(x, *popt), 'r-', label="Fitted Curve")
ax.legend(loc='lower right')

sns.despine()
ax.set_title('Fit to model attraction when Ps1(agent 1) = 0')
ax.set_xlabel('Ps2 (agent 2)')
ax.set_ylabel('Model attraction')
print(popt)

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))

nSteps=10000
xSteps=np.linspace(0,1,nSteps)
ssInterpolated=np.zeros((8,nSteps))

poptAll=[]

for i in range(8):
    x = dfAvgPiv.columns
    yn = dfAvgPiv.values[i,:]

    popt, pcov = curve_fit(BrodyGrowth, x, yn)
    poptAll.append(popt)
    plt.plot(x, yn, '.-', color='gray')
    ssInterpolated[i,:]=BrodyGrowth(xSteps, *popt)
    plt.plot(xSteps, ssInterpolated[i,:], 'r-')
    xy=np.linspace(0,1,50)
    print(popt)
    

ax.set_ylim([-.05,1.05])   


sns.despine()
ax.set_title('Fit to model attraction for various Ps1(agent 1)')
ax.set_xlabel('Ps2 (agent 2)')
ax.set_ylabel('Model attraction')



In [ ]:
import scipy.stats
from scipy.interpolate import interp1d

poptAll=np.array(poptAll)
poptAll
yLimAll=[1,1,5]
x = dfAvgPiv.index
titles=np.array(['a:max','b:start','c:slope'])

fig, axes = plt.subplots(nrows=1, ncols=3,figsize=(10, 3.3),sharex=True)

for i in range(3):
    axes[i].plot(x,poptAll[:,i],'o-k')
    axes[i].set_ylim([0,yLimAll[i]])    
    axes[i].set_title(titles[i])
    if i==1:
        axes[i].set_xlabel('Ps2')

y=poptAll[:,i]
t=np.linspace(0,1,101)
f = interp1d(x, y)
curvInt=f(t)
axes[i].plot(t,curvInt,'.',color='gray',ms=4)



# now use popt parameter from condition Ps1=0 to predict popt parameter for conditions Ps1>0

use:

    a = poptPs10[0]      # implies maximal attraction is independent of Ps1
    b = func(poptPs10)   # use attraction when Ps1==0 & Ps2>x as start for curve when Ps1==x
    c = curvInt(x)       # read slope from interpolated empirical function c(Ps2)

In [ ]:
fig, ax = plt.subplots(figsize=(9/inToCm,9/inToCm))


for i in range(len(dfAvgPiv.index)):
    try:
        xo=dfAvgPiv.columns
        yo = dfAvgPiv.values[i,:]
    except:
        pass
        
    x = np.linspace(0,1,50)#
    c=curvInt[(dfAvgPiv.index[i]/.01).astype('int')]
    popt = (poptPs10[0],(BrodyGrowth(dfAvgPiv.index,*(poptPs10))[i]),c)

    plt.plot(xo, yo, '.-', color='gray')
    plt.plot(x, BrodyGrowth(x, *popt), 'r-')
    xy=np.linspace(0,1,50)

plt.ylabel('Model attraction')
plt.xlabel('Social drive animal 1')
sns.despine()

# To run the NetLogo model using 15 agents that represent the social attraction observed in the real animals, infer Ps for each animal by looking up Ps1 for model attraction == unilateral attraction of real animal

In [ ]:
xfine=np.linspace(0,1,10000)
curveFine=BrodyGrowth(xfine, *poptPs10)
ySteps=np.arange(0,.8,.01)
xSteps=[]
for i in uniLateral:
    tmp=np.where(curveFine>i)[0][0]/10000.
    xSteps.append("{:.4f}".format(tmp))

' '.join(xSteps[::-1])

## Use these Ps values to run NetLogo model!

In [ ]:
csvFileNetLogo=os.path.join(RawDataDir,'20180316_netlogo2Animals\\Shoaling paperRun_FineGrainRealPs_review-table.csv')

df_NL=pd.read_csv(csvFileNetLogo,skiprows=6)
print('Ps values found: ' ,df_NL['t1-Social'].unique())
df_NL=df_NL.groupby(['[run number]','t1-Social','t2-Social']).mean().IAD.reset_index() #drop other columns

#calculate shoaling index, using IAD when both agents have Ps<0.01
idx=(df_NL['t1-Social']<0.001) & (df_NL['t2-Social']<0.001) 
IADs=df_NL[idx].IAD
print('mean IAD without attraction: ',IADs.mean())
df_NL['SI']=(IADs.mean()-df_NL.IAD)/IADs.mean()
df_NL.head()

In [ ]:
df_NL['a1']=BrodyGrowth(df_NL['t1-Social'], *poptPs10)
df_NL['a2']=BrodyGrowth(df_NL['t2-Social'], *poptPs10)
df_NL['linPredict']=df_NL['a1']+df_NL['a2']

fig, ax = plt.subplots(figsize=(4,4))
idx=(df_NL['a1']<0.8) & (df_NL['a2']<0.8)

plt.scatter(df_NL[idx]['linPredict'],df_NL[idx]['SI'],1)
plt.plot(np.arange(0,1,.1),np.arange(0,1,.1),'r',lw=3,label='unity')

plt.ylabel('Model attraction')
plt.xlabel('Linear combination prediction \n (unilateral #1 + #2)')

In [ ]:
dfTmp=df_NL.copy()

aa=dfTmp[['t1-Social','t2-Social']].values
dfTmp.loc[:,['t1-Social','t2-Social']]=np.sort(aa,axis=1)[:,::-1]


dfTmp['a1']=BrodyGrowth(dfTmp['t1-Social'], *poptPs10)
dfTmp['a2']=BrodyGrowth(dfTmp['t2-Social'], *poptPs10)
dfTmp['linPredict']=dfTmp['a1']+dfTmp['a2']

dfTmp=dfTmp.sort_values('a1', ascending=False)

df2=dfTmp.groupby(['t1-Social','t2-Social']).mean().reset_index()
df2=df2.sort_values(['t1-Social','t2-Social'], ascending=False)

selfPair=df2['t1-Social'].values==df2['t2-Social'].values
df2=df2[~selfPair]
df2.shape


In [ ]:
import scipy.stats
pfp.paper()
inToCm=2.54
plt.figure(figsize=(4.5/inToCm,4.5/inToCm))
ax = plt.gca()

idx=(df2['a1']<0.8) & (df2['a2']<0.8)

plt.scatter(df2[idx]['linPredict'],df2[idx]['SI'],color='gray',s=6)
#plt.plot(np.arange(0,1,.1),np.arange(0,1,.1),'r',lw=3,label='unity')

plt.ylabel('Model attraction')
plt.xlabel('Linear combination prediction \n (unilateral #1 + #2)')

sns.despine()

s,i,r,p,std=scipy.stats.linregress(df2[idx]['linPredict'],df2[idx]['SI'])
print(s,i,r,p,std)
t=np.linspace(0,1,100)
l=i+s*t
ax.plot(t,l,ls='--',color='k',lw=2)
plt.plot(np.arange(0,1,.1),np.arange(0,1,.1),'gray',ls=':')
ax.text(.1,.9,'R: '+str(r)[:4])
ax.text(.1,.8,"p = {:.0e}".format(p),color='k')
ax.text(.1,.7,"slope = {:.2f}".format(s),color='k')

ax.set_ylim([0,1])
ax.set_xlim([0,1])
ax.set_yticks([0,.5,1])
ax.set_xticks([0,.5,1]);

figPath=outputDir+'\\4F_Model_LinearCombinationsRealPs.svg'
plt.savefig(figPath,bbox_inches='tight')

In [ ]:
pfp.paper()
inToCm=2.54
plt.figure(figsize=(4.5/inToCm,4.5/inToCm))
ax = plt.gca()

x=df2['SI']
y=simArray[:,3]

plt.scatter(x,y,color='gray',s=6)
#plt.plot(np.arange(0,1,.1),np.arange(0,1,.1),'r',lw=3,label='unity')

plt.ylabel('Data attraction')
plt.xlabel('Model attraction')

sns.despine()

s,i,r,p,std=scipy.stats.linregress(x,y)
print(s,i,r,p,std)
t=np.linspace(0,1,100)
l=i+s*t
ax.plot(t,l,ls='--',color='k',lw=2)
plt.plot(np.arange(0,1,.1),np.arange(0,1,.1),'gray',ls=':')
ax.text(.1,.9,'R: '+str(r)[:4])
ax.text(.1,.8,"p = {:.0e}".format(p),color='k')
ax.text(.1,.7,"slope = {:.2f}".format(s),color='k')

ax.set_ylim([0,1])
ax.set_xlim([0,1])
ax.set_yticks([0,.5,1])
ax.set_xticks([0,.5,1]);

from shutil import copy2

def splitall(path):
    allparts = []
    while 1:
        parts = os.path.split(path)
        if parts[0] == path:  # sentinel for absolute paths
            allparts.insert(0, parts[0])
            break
        elif parts[1] == path: # sentinel for relative paths
            allparts.insert(0, parts[1])
            break
        else:
            path = parts[0]
            allparts.insert(0, parts[1])
    return allparts



for i,row in info.iterrows():
    fn=row.txtPath
    head, tail = os.path.split(fn)

    copyList=[]
    copyList.append(glob.glob(head+'\\ROI*.csv')[0])
    copyList.append(glob.glob(head+'\\PositionTxt*.txt')[0])
    copyList.append(glob.glob(head+'\\PL*.txt')[0])
    copyList.append(glob.glob(head+'\\*anSize.csv')[0])
    
    for f in copyList:
        print(f)
        if f[0]=='E':
            keepSlash=3
        else:
            keepSlash=4
        toDirectory = "e:\\b\\LarschAndBaier2018\\RawData\\" + os.path.join(*splitall(f)[keepSlash:-1])+"\\"
        #toDirectory = "e:\\b\\LarschAndBaier2018\\RawData\\" 
        if not os.path.isdir(toDirectory):
            os.makedirs(toDirectory)
        
        copy2(f, toDirectory)
